In [152]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re
import numpy as np
import os
from typing import List, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
from fastparquet import ParquetFile, write

In [63]:
base_url = 'https://th.sportscorpion.com/eng/tournament/archive/?page='

# Create a set to store the URLs
all_urls = set()

# Load the existing URLs from the file if it exists
if os.path.exists('tournament_urls.txt'):
    with open('tournament_urls.txt', 'r') as f:
        all_urls.update(line.strip() for line in f)

In [64]:
# Start from page 1
page_num = 1

new_urls = set()

while page_num < 10: # adjust this if it is a long time since the script was run
    # Get the HTML content of the page
    response = requests.get(base_url + str(page_num))

    # If the page doesn't exist, break the loop
    if response.status_code != 200:
        break

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table in the HTML
    table = soup.find('table', {'class': 'sTable'})

    # If there's no table on the page, break the loop
    if table is None:
        break

    # Find all 'a' tags within the table
    links = table.find_all('a')

    # Extract the href attribute from each 'a' tag, but only if the last character is a digit
    urls = {link.get('href') for link in links if link.get('href')[-2].isdigit()}

    # Check if any of the URLs is already in the set
    if any(url in all_urls for url in urls):
        # If any of the URLs is already in the set, remove the urls that are in the set
        urls = {url for url in urls if url not in all_urls}
    
    # Add the new URLs to the file
    with open('tournament_urls.txt', 'a') as f:
        for url in urls:
            f.write(url + '\n')

    # Add the URLs to the set
    all_urls.update(urls)
    new_urls.update(urls)
    # Go to the next page
    page_num += 1

# Now new_urls contains all the new unique URLs
print(f"Collected {len(new_urls)} new unique URLs.")


Collected 7 new unique URLs.


In [65]:
len(all_urls)

5490

In [66]:
len(new_urls)

7

In [97]:
BASE_URL = "https://th.sportscorpion.com"

# Mapping of playoff stage names to fraction values
PLAYOFF_STAGE_MAP = {
    "1/64 final": 1/64,
    "1/32 final": 1/32,
    "1/16 final": 1/16,
    "1/8 final": 1/8,
    "Quarterfinal": 1/4,
    "Semi-final": 1/2,
    "Final": 1,
    "Match for the third place": 0.9
}

def fetch_page(session, url: str) -> BeautifulSoup:
    # Fetch the page content and return a BeautifulSoup object
    response = session.get(url)
    return BeautifulSoup(response.text, 'lxml')

def clean_score_text(score_text: str) -> str:
    """
    Clean the raw score text by removing OT, W.O, etc.
    """
    # A single chain of replaces is slightly more efficient than repeated calls
    # Also more readable and consistent
    return (score_text
            .replace('(OT)', '')
            .replace('(W.O)', '')
            .replace('\xa0', '')
            .replace('*', '')
            .replace('\n', ''))

def get_playoff_stage_fraction(stage_name: str) -> float:
    # Convert the playoff stage name into a numeric fraction, e.g. 'Quarterfinal' -> 0.25
    stage_name = stage_name.strip().lower()
    for key, val in PLAYOFF_STAGE_MAP.items():
        if key.lower() in stage_name:
            return val
    # If not found in map, default to None (unknown stage)
    return None

def extract_name_and_id(a_tag) -> Tuple[str, str]:
    """
    Given an <a> tag for a player, return (player_name, player_id).
    If there's no <a> tag or it's missing an ID, returns (None, None).
    """
    if not a_tag:
        return None, None
    
    player_name = a_tag.text.strip()
    href = a_tag.get('href', '')
    match_id = re.search(r'/user/id/(\d+)/', href)
    player_id = match_id.group(1) if match_id else None
    
    return player_name, player_id

def get_match_info(session, url: str) -> List[Tuple[str, str, str, str, str, int, int, str, str, float, int]]:
    """
    Fetch match information from a given stage page.
    
    Returns a list of tuples:
    (
        URL,
        Player1Name,
        Player1ID,
        Player2Name,
        Player2ID,
        GoalsPlayer1,
        GoalsPlayer2,
        Overtime,
        Stage,            # 'Playoff' or 'Round-Robin'
        RoundNumber,      # numeric fraction for playoff stage or round # for RR
        PlayoffGameNumber # int or None
    )
    """
    soup = fetch_page(session, url)
    
    # Remove the 'saved-matches' section to avoid duplicates
    saved_matches_div = soup.find('div', class_='saved-matches')
    if saved_matches_div:
        saved_matches_div.decompose()
    
    match_info = []

    # Check if the page is for the playoff stage
    is_playoff = len(soup.select('tr.series-container')) > 0

    if is_playoff:
        # For playoff stages, matches are organized by subheaders (Quarterfinal, Semi-final, etc.)
        subheaders = soup.select('div.subheader')
        for subheader in subheaders:
            stage_name = subheader.get_text(strip=True)
            playoff_fraction = get_playoff_stage_fraction(stage_name)

            # Find the .gr_match blocks after this subheader until next subheader
            next_siblings = subheader.find_all_next('div', class_='gr_match')
            for block in next_siblings:
                # If block belongs to another subheader, stop processing further blocks
                next_sub = block.find_previous_sibling('div', class_='subheader')
                if next_sub and next_sub != subheader:
                    break

                series = block.select('tr.series-container')
                for serie in series:
                    # Each player is in 'td[class^="ma_name"] a'
                    # We want the first link for player1, second link for player2
                    players = serie.select('td[class^="ma_name"] a')
                    if len(players) < 2:
                        continue
                    player1_name, player1_id = extract_name_and_id(players[0])
                    player2_name, player2_id = extract_name_and_id(players[1])
                    
                    # Each 'td[class^="ma_result_"]' corresponds to one game in the series
                    scores = serie.select('td[class^="ma_result_"]')
                    # Ignore the last score which is the total series score
                    for game_number, score in enumerate(scores[:-1], start=1):
                        if ':' in score.text:
                            score_cleaned = clean_score_text(score.text)
                            try:
                                goals_player_1, goals_player_2 = map(int, score_cleaned.split(':'))
                                overtime = 'Yes' if '(OT)' in score.text else 'No'
                                match_info.append(
                                    (
                                        url,
                                        player1_name,
                                        player1_id,
                                        player2_name,
                                        player2_id,
                                        goals_player_1,
                                        goals_player_2,
                                        overtime,
                                        'Playoff',
                                        playoff_fraction,
                                        game_number
                                    )
                                )
                            except ValueError:
                                continue

    else:
        # Scrape round-robin matches
        match_tables = soup.select('table.grTable')
        for table in match_tables:
            header = table.select_one('th:-soup-contains("Tour")')
            if header:
                round_text = header.get_text(strip=True)
                round_match = re.search(r'(\d+)\s*Tour', round_text)
                round_number = float(round_match.group(1)) if round_match else None
            else:
                round_number = None

            rows = table.select('tr[id^="match"]')
            for row in rows:
                # Player 1 link
                player1_a = row.select_one('td.ma_name1 a')
                player1_name, player1_id = extract_name_and_id(player1_a)
                # If no <a>, fallback to the raw text
                if not player1_name:
                    player1_name = row.select_one('td.ma_name1').text.strip()

                # Player 2 link
                player2_a = row.select_one('td.ma_name2 a')
                player2_name, player2_id = extract_name_and_id(player2_a)
                # If no <a>, fallback to the raw text
                if not player2_name:
                    player2_name = row.select_one('td.ma_name2').text.strip()

                score = row.select_one('td[class^="ma_result_"]')
                if score and ':' in score.text:
                    score_cleaned = (
                        score.text.replace('(OT)', '')
                                  .replace('(W.O)', '')
                                  .replace('\xa0', '')
                                  .replace('*', '')
                                  .replace('\n', '')
                    )
                    try:
                        goals_player_1, goals_player_2 = map(int, score_cleaned.split(':'))
                        overtime = 'Yes' if '(OT)' in score.text else 'No'
                        # For round-robin, we don't have a playoff game number, so set None
                        match_info.append(
                            (
                                url,
                                player1_name,
                                player1_id,
                                player2_name,
                                player2_id,
                                goals_player_1,
                                goals_player_2,
                                overtime,
                                'Round-Robin',
                                round_number,
                                None  # <-- None for round-robin
                            )
                        )
                    except ValueError:
                        print(f"Unable to parse score '{score_cleaned}' from match {url}")

    return match_info

def get_tournament_matches(tournament_urls: List[str], existing_stage_ids: set[str]) -> pd.DataFrame:
    all_matches = []
    headers = {'User-Agent': 'Mozilla/5.0'}

    def fetch_tournament_data(url):
        with requests.Session() as session:
            session.headers.update(headers)
            tournament_id = url.split('/')[-2]
            tournament_url = f"{BASE_URL}/eng/tournament/id/{tournament_id}/"
            tournament_soup = fetch_page(session, tournament_url)

            # Check if tournament is a team tournament, skip if yes
            tournament_type_element = tournament_soup.select_one("th:-soup-contains('Tournament type') + td")
            tournament_type = tournament_type_element.text.strip() if tournament_type_element else 'Unknown'
            if tournament_type.lower() == 'team':
                return []

            # Extract tournament name and date
            tournament_name_element = tournament_soup.select_one("h1#header")
            tournament_name = tournament_name_element.text.strip() if tournament_name_element else 'Unknown'

            date_element = tournament_soup.select_one("th:-soup-contains('Date of the tournament') + td")
            date = date_element.text.strip() if date_element else 'Unknown'

            # Extract the stages and their sequences
            stage_rows = tournament_soup.select('table.stages-table tr')
            stage_data = []
            for row in stage_rows:
                seq_cell = row.select_one('td.stage-gr')
                if seq_cell:
                    stage_sequence = seq_cell.get_text(strip=True)
                    sched_link = row.select_one('a:-soup-contains("Schedule and results")')
                    if sched_link:
                        stage_url = f"{BASE_URL}{sched_link['href']}?print"
                        stage_id = stage_url.split('/')[-3]
                        stage_data.append((stage_id, stage_url, stage_sequence))

            stage_matches = []
            for stage_id, stage_url, stage_sequence in stage_data:
                # Skip stage if already in existing_stage_ids
                if stage_id in existing_stage_ids:
                    continue
                matches = get_match_info(session, stage_url)
                for match in matches:
                    # match = (
                    #   url,
                    #   Player1Name, Player1ID,
                    #   Player2Name, Player2ID,
                    #   GoalsPlayer1, GoalsPlayer2,
                    #   Overtime, Stage, RoundNumber,
                    #   PlayoffGameNumber
                    # )
                    stage_matches.append((
                        int(stage_id) if stage_id.isdigit() else None,  # Ensure StageID is numeric
                        match[1],  # Player1Name
                        int(match[2]) if match[2] and match[2].isdigit() else None,  # Player1ID
                        match[3],  # Player2Name
                        int(match[4]) if match[4] and match[4].isdigit() else None,  # Player2ID
                        match[5],  # GoalsPlayer1
                        match[6],  # GoalsPlayer2
                        match[7],  # Overtime
                        match[8],  # Stage
                        match[9],  # RoundNumber
                        match[10],  # PlayoffGameNumber
                        date,
                        tournament_name,
                        int(tournament_id),  # Ensure TournamentID is numeric
                        int(stage_sequence) if stage_sequence.isdigit() else None  # Ensure StageSequence is numeric
                    ))
            return stage_matches

    processed_tournaments = 0

    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(fetch_tournament_data, url): url for url in tournament_urls}
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url),
                           desc="Processing tournaments", unit="tournament"):
            url = future_to_url[future]
            try:
                matches = future.result()
                all_matches.extend(matches)
                processed_tournaments += 1
                tqdm.write(f"\rProcessed tournaments: {processed_tournaments}", end='')
            except Exception as exc:
                print(f'{url} generated an exception: {exc}')

    # Build DataFrame
    df = pd.DataFrame(
        all_matches, 
        columns=[
            'StageID',
            'Player1',
            'Player1ID',
            'Player2',
            'Player2ID',
            'GoalsPlayer1',
            'GoalsPlayer2',
            'Overtime',
            'Stage',
            'RoundNumber',
            'PlayoffGameNumber',
            'Date',
            'TournamentName',
            'TournamentID',
            'StageSequence'
        ]
    )

    # Convert columns to match the schema
    df['StageID'] = pd.to_numeric(df['StageID'], errors='coerce', downcast='integer')
    df['Player1ID'] = pd.to_numeric(df['Player1ID'], errors='coerce', downcast='integer')
    df['Player2ID'] = pd.to_numeric(df['Player2ID'], errors='coerce', downcast='integer')
    df['TournamentID'] = pd.to_numeric(df['TournamentID'], errors='coerce', downcast='integer')
    df['StageSequence'] = pd.to_numeric(df['StageSequence'], errors='coerce', downcast='integer')

    # Format 'Date' to string for Parquet compatibility
    df['Date'] = df['Date'].apply(
        lambda x: pd.to_datetime(x, format='%d.%m.%Y', errors='coerce').strftime('%Y-%m-%d') if pd.notnull(x) else None
    )


    # Sort data
    df.sort_values(by=["Date", "StageSequence", "RoundNumber", "PlayoffGameNumber"], 
                   inplace=True, na_position='last')

    # Remove playoff draws
    df = df[~((df['Stage'] == 'Playoff') & (df['GoalsPlayer1'] == df['GoalsPlayer2']))]
    df.reset_index(drop=True, inplace=True)

    return df


In [98]:
df = get_tournament_matches(list(new_urls), existing_stage_ids=set())

Processing tournaments:   0%|          | 0/7 [00:00<?, ?tournament/s]

Processed tournaments: 7

In [168]:
file_path = 'th_matches.parquet'

if os.path.exists(file_path):
    # Read existing data
    existing_df = pd.read_parquet(file_path, engine='fastparquet')
    # Concatenate with the new data
    combined_df = pd.concat([existing_df, df], ignore_index=True)

# Write the combined DataFrame back to Parquet with compression
combined_df.to_parquet(file_path, engine='fastparquet', compression='zstd')

## Create Player table with unique IDs

In [153]:
df = pd.read_parquet('th_matches.parquet', engine='fastparquet')

# Extract unique players from Player1 and Player2 columns
player1_data = df[['Player1ID', 'Player1']].rename(columns={'Player1ID': 'PlayerID', 'Player1': 'PlayerName'})
player2_data = df[['Player2ID', 'Player2']].rename(columns={'Player2ID': 'PlayerID', 'Player2': 'PlayerName'})

# Combine both datasets
players_data = pd.concat([player1_data, player2_data])

# Drop duplicates to ensure unique PlayerID and PlayerName
scorpion_players = players_data.drop_duplicates().reset_index(drop=True)

# Ensure PlayerID is an integer
scorpion_players['PlayerID'] = scorpion_players['PlayerID'].astype(int)

# sort by PlayerID
scorpion_players = scorpion_players.sort_values(by='PlayerID').reset_index(drop=True)

## Get ITHF WR ID for each player if available

In [154]:
# Define the base URL
BASE_URL = "https://th.sportscorpion.com"

# Function to fetch the WR ID
def fetch_wr_id(player_id):
    url = f"{BASE_URL}/eng/user/id/{player_id}/"
    try:
        # Use a session for persistent connections
        with requests.Session() as session:
            response = session.get(url, headers={"User-Agent": "Mozilla/5.0"})
            if response.status_code == 200:
                # Directly parse the table containing WR ID
                soup = BeautifulSoup(response.text, "lxml")
                table = soup.find("table", class_="iTable")
                if table:
                    # Search for the WR ID using regex in the table text
                    match = re.search(r'ID <a.*?ID=(\d+)', str(table))
                    if match:
                        return match.group(1)  # Extract WR ID
    except Exception as e:
        tqdm.write(f"Error fetching WR ID for PlayerID {player_id}: {e}")
    return None  # Return None if not found or an error occurs

# Fetch WR IDs concurrently using ThreadPoolExecutor
def fetch_all_wr_ids(player_ids):
    wr_ids = []
    with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
        # Use tqdm for a progress bar
        results = list(tqdm(executor.map(fetch_wr_id, player_ids), total=len(player_ids), desc="Fetching WR IDs"))
        wr_ids.extend(results)
    return wr_ids

# Add the WR ID column
scorpion_players['WR_ID'] = fetch_all_wr_ids(scorpion_players["PlayerID"])

Fetching WR IDs:   0%|          | 0/9334 [00:00<?, ?it/s]